In [2]:
import mysql.connector
from mysql.connector import errorcode
import pandas as pd
from sqlalchemy import create_engine, Column, Integer, String, ForeignKey, Date , Float, Table, Text, SmallInteger, UniqueConstraint,func
from sqlalchemy.orm import declarative_base, relationship, sessionmaker
import pandas as pd
import ast
import logging
from datetime import datetime
from sqlalchemy.engine import URL
from urllib.parse import quote_plus
import unicodedata
from rapidfuzz import process, fuzz


In [3]:
conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password="Physic955"
)
cursor = conn.cursor()

cursor.execute("CREATE DATABASE IF NOT EXISTS Basketball_Reference")
print("Database created (or already exists).")

conn.close()

Database created (or already exists).


In [4]:
DB_NAME = "Basketball_Reference"

config = {
  'host': 'localhost',
  'user': 'root',
  'password': 'Physic955', 
  'database' : DB_NAME
  
}


print("Connection parameters defined.")

Connection parameters defined.


In [5]:

df_players = pd.read_csv("../Phase_1_Data extraction/players_info.csv")

df_players['PlayerID'] = range(1, len(df_players) + 1)
df_players_clean = df_players.drop(columns=['Positions', 'Current_team'])

positions_set = set()
for pos_list in df_players['Positions']:
    if pd.notna(pos_list) and pos_list.strip() not in ["", "[]"]:
        for pos in eval(pos_list):
            if pos.strip() != "":
                positions_set.add(pos)

df_positions = pd.DataFrame({
    'PositionID': range(1, len(positions_set) + 1),
    'PositionName': list(positions_set)
})

player_positions = []
for _, row in df_players.iterrows():
    player_id = row['PlayerID']
    pos_list = row['Positions']
    if pd.notna(pos_list) and pos_list.strip() not in ["", "[]"]:
        for pos in eval(pos_list):
            if pos.strip() != "":
                pos_id = df_positions[df_positions['PositionName'] == pos]['PositionID'].values[0]
                player_positions.append({'PlayerID': player_id, 'PositionID': pos_id})

df_player_positions = pd.DataFrame(player_positions)


teams_unique = df_players['Current_team'].dropna().unique()
df_teams = pd.DataFrame({
    'TeamID': range(1, len(teams_unique) + 1),
    'TeamName': teams_unique
})


player_teams = []
for _, row in df_players.iterrows():
    player_id = row['PlayerID']
    team_name = row['Current_team']
    if pd.notna(team_name):
        team_id = df_teams[df_teams['TeamName'] == team_name]['TeamID'].values[0]
        player_teams.append({'PlayerID': player_id, 'TeamID': team_id})

df_player_teams = pd.DataFrame(player_teams)

df_players_clean.to_excel("Players.xlsx", index=False)                
df_positions.to_excel("Positions.xlsx", index=False)                 
df_player_positions.to_excel("Player_Positions.xlsx", index=False)    
df_teams.to_excel("Teams.xlsx", index=False)                          
df_player_teams.to_excel("Player_Teams.xlsx", index=False)            


In [6]:
df1 = pd.read_excel("Teams.xlsx")
df2 = pd.read_csv("../Phase_1_Data extraction/team_infos.csv")
df_merged = pd.merge(df1, df2, left_on="TeamName", right_on="Team", how="left")
df_merged.drop(columns=["Team"], inplace=True)
df_merged.to_excel("Teams_Final.xlsx", index=False)

In [7]:

def normalize_name(name):
    if pd.isna(name):
        return ""
    return unicodedata.normalize('NFKD', str(name)).encode('ascii', 'ignore').decode('ascii').strip().lower()


df_team  = pd.read_excel("Teams_Final.xlsx")       
df_players = pd.read_excel("Players.xlsx")
df_champions = pd.read_csv("../Phase_1_Data extraction/champions.csv")

df_players['Name_norm'] = df_players['Name'].apply(normalize_name)
players_list_norm = df_players['Name_norm'].tolist()

df_champions.columns = df_champions.columns.str.strip()  
seasons_unique = df_champions['season'].drop_duplicates().reset_index(drop=True)
df_seasons = pd.DataFrame({
    'SeasonID': range(1, len(seasons_unique)+1),
    'Season': seasons_unique
})
df_champions = df_champions.merge(df_seasons, left_on='season', right_on='Season', how='left')


champions_unique = df_champions['Champion'].drop_duplicates().reset_index(drop=True)
df_champion_name = pd.DataFrame({'ChampionName': champions_unique})


df_champion_id = df_champion_name.merge(
    df_team[['TeamName', 'TeamID']],
    left_on='ChampionName',
    right_on='TeamName',
    how='left'
)


df_champion_id['ChampionID'] = range(1, len(df_champion_id)+1)
df_champion_id = df_champion_id[['ChampionID', 'ChampionName', 'TeamID']]

df_champions = df_champions.merge(df_champion_id, left_on='Champion', right_on='ChampionName', how='left')

rows = []
for idx, row in df_champions.iterrows():
    season_id = row['SeasonID']
    champion_id = row['ChampionID']
    players_str = row['players']
    
    try:
        players_list = ast.literal_eval(players_str)
    except:
        players_list = []
    
    for player_name in players_list:
        player_name_norm = normalize_name(player_name)
        match, score, match_idx = process.extractOne(player_name_norm, players_list_norm, scorer=fuzz.ratio)
        
        if score >= 80:  
            player_id = df_players.iloc[match_idx]['PlayerID']
            rows.append({
                'SeasonID': season_id,
                'ChampionID': champion_id,
                'PlayerID': player_id
            })
        else:
            print(f"Player not found (low similarity): {player_name}")

df_final_players = pd.DataFrame(rows)


df_seasons.to_excel("seasons.xlsx", index=False)
df_champion_id.to_excel("Champions_ID.xlsx", index=False)
df_final_players.to_excel("Champions_Players.xlsx", index=False)


In [8]:


def normalize_name(name):
    if pd.isna(name):
        return ""
    return unicodedata.normalize('NFKD', str(name)).encode('ascii', 'ignore').decode('ascii').strip().lower()


df_players = pd.read_excel("Players.xlsx") 
df_seasons = pd.read_excel("Seasons.xlsx")  


df_players['Name_norm'] = df_players['Name'].apply(normalize_name)
players_list_norm = df_players['Name_norm'].tolist()


df_ranks = pd.read_csv("../Phase_1_Data extraction/players_ranked.csv")
df_ranks['name_norm'] = df_ranks['name'].apply(normalize_name)


df_ranks = df_ranks.merge(df_seasons, left_on='season', right_on='Season', how='left')


player_ids = []
for name in df_ranks['name_norm']:
    match, score, idx = process.extractOne(name, players_list_norm, scorer=fuzz.ratio)
    if score >= 80:
        player_ids.append(df_players.iloc[idx]['PlayerID'])
    else:
        player_ids.append(None)
        print(f"Player not found (low similarity): {name}")

df_ranks['PlayerID'] = player_ids


df_final_ranks = df_ranks[['rank', 'PlayerID', 'SeasonID']]


df_final_ranks.to_excel("Player_Ranks.xlsx", index=False)



In [9]:
def normalize_name(name):
    if pd.isna(name):
        return ""
    return unicodedata.normalize('NFKD', str(name)).encode('ascii', 'ignore').decode('ascii').strip().lower()


df_players = pd.read_excel("Players.xlsx") 
df_seasons = pd.read_excel("Seasons.xlsx")   

df_players['Name_norm'] = df_players['Name'].apply(normalize_name)
players_list_norm = df_players['Name_norm'].tolist()

df_ranks = pd.read_csv("../Phase_1_Data extraction/Michael_Jordan_Trophy.csv")
df_ranks['name_norm'] = df_ranks['name'].apply(normalize_name)

df_ranks = df_ranks.merge(df_seasons, left_on='season', right_on='Season', how='left')


player_ids = []
for name in df_ranks['name_norm']:
    match, score, idx = process.extractOne(name, players_list_norm, scorer=fuzz.ratio)
    if score >= 65:
        player_ids.append(df_players.iloc[idx]['PlayerID'])
    else:
        player_ids.append(None)
        print(f"Player not found (low similarity): {name}")

df_ranks['PlayerID'] = player_ids

df_final_ranks = df_ranks[['rank', 'PlayerID', 'SeasonID']]

df_final_ranks.to_excel("Michael_Jordan_Trophy.xlsx", index=False)

In [10]:
Base = declarative_base()

class Player(Base):
    __tablename__ = "players"

    PlayerID = Column(Integer, primary_key=True, autoincrement=True)
    Name = Column(String(100), nullable=False)
    Birth_date = Column(Date, nullable=False)
    Age = Column(Integer)
    Status = Column(String(20))
    Height = Column(Integer)
    Weight = Column(Integer)
    Nationality = Column(String(50))
    Shoots = Column(String(10))
    Experience = Column(Integer)
    PTS = Column(Float)


    player_positions = relationship("PlayerPositions", back_populates="player", cascade="all, delete-orphan")
    player_teams = relationship("PlayerTeam", back_populates="player", cascade="all, delete-orphan")
    champion_players = relationship("ChampionPlayers", back_populates="player", cascade="all, delete-orphan")
    player_ranks = relationship("PlayerRanks", back_populates="player", cascade="all, delete-orphan")
    michael_jordan_trophies = relationship("MichaelJordanTrophy", back_populates="player", cascade="all, delete-orphan")

class Position(Base):
    __tablename__ = "positions"

    PositionID = Column(Integer, primary_key=True, autoincrement=True)
    PositionName = Column(String(50), unique=True)


    player_positions = relationship("PlayerPositions", back_populates="position", cascade="all, delete-orphan")

class PlayerPositions(Base):
    __tablename__ = "player_positions"

    PlayerID = Column(Integer, ForeignKey("players.PlayerID"), primary_key=True)
    PositionID = Column(Integer, ForeignKey("positions.PositionID"), primary_key=True)

    player = relationship("Player", back_populates="player_positions")
    position = relationship("Position", back_populates="player_positions")


class Team(Base):
    __tablename__ = "teams"

    TeamID = Column(Integer, primary_key=True, autoincrement=True)
    TeamName = Column(String(100), unique=True)
    Location = Column(String(100))
    Founded = Column(String(20))  
    Championships = Column(Integer)
    Playoff_Appearances = Column(Integer)


    player_teams = relationship("PlayerTeam", back_populates="team", cascade="all, delete-orphan")

class PlayerTeam(Base):
    __tablename__ = "player_teams"

    PlayerID = Column(Integer, ForeignKey("players.PlayerID"), primary_key=True)
    TeamID = Column(Integer, ForeignKey("teams.TeamID"), primary_key=True)

    player = relationship("Player", back_populates="player_teams")
    team = relationship("Team", back_populates="player_teams")


class Season(Base):
    __tablename__ = "seasons"

    SeasonID = Column(Integer, primary_key=True, autoincrement=True)
    Season = Column(String(20), unique=True)


    champion_players = relationship("ChampionPlayers", back_populates="season", cascade="all, delete-orphan")
    player_ranks = relationship("PlayerRanks", back_populates="season", cascade="all, delete-orphan")
    michael_jordan_trophies = relationship("MichaelJordanTrophy", back_populates="season", cascade="all, delete-orphan")


class Champion(Base):
    __tablename__ = "champions"

    ChampionID = Column(Integer, primary_key=True, autoincrement=True)
    ChampionName = Column(String(100), unique=True)
    TeamID = Column(Integer, ForeignKey("teams.TeamID"))  

    champion_players = relationship("ChampionPlayers", back_populates="champion", cascade="all, delete-orphan")
    team = relationship("Team")


class ChampionPlayers(Base):
    __tablename__ = "champion_players"

    SeasonID = Column(Integer, ForeignKey("seasons.SeasonID"), primary_key=True)
    ChampionID = Column(Integer, ForeignKey("champions.ChampionID"), primary_key=True)
    PlayerID = Column(Integer, ForeignKey("players.PlayerID"), primary_key=True)

    champion = relationship("Champion", back_populates="champion_players")
    player = relationship("Player", back_populates="champion_players")
    season = relationship("Season", back_populates="champion_players")


class PlayerRanks(Base):
    __tablename__ = "player_ranks"

    RankID = Column(Integer, primary_key=True, autoincrement=True)
    Rank = Column(Integer, nullable=False)
    PlayerID = Column(Integer, ForeignKey("players.PlayerID"), nullable=False)
    SeasonID = Column(Integer, ForeignKey("seasons.SeasonID"), nullable=False)

    player = relationship("Player", back_populates="player_ranks")
    season = relationship("Season", back_populates="player_ranks")


class MichaelJordanTrophy(Base):
    __tablename__ = "michael_jordan_trophy"

    TrophyID = Column(Integer, primary_key=True, autoincrement=True)
    Rank = Column(Integer, nullable=False)
    PlayerID = Column(Integer, ForeignKey("players.PlayerID"), nullable=False)
    SeasonID = Column(Integer, ForeignKey("seasons.SeasonID"), nullable=False)

    player = relationship("Player", back_populates="michael_jordan_trophies")
    season = relationship("Season", back_populates="michael_jordan_trophies")


In [11]:
password = quote_plus("Physic955")

engine = create_engine(
    f"mysql+mysqlconnector://{config['user']}:{password}@{config['host']}/{config['database']}"
)

Base.metadata.create_all(engine)

In [12]:
df_players = pd.read_excel("Players.xlsx")         
df_positions = pd.read_excel("positions.xlsx")       
df_teams = pd.read_excel("Teams_Final.xlsx")               
df_player_positions = pd.read_excel("player_positions.xlsx")  
df_player_teams = pd.read_excel("player_teams.xlsx")          
df_seasons = pd.read_excel("Seasons.xlsx")  
df_champions = pd.read_excel("Champions_ID.xlsx")  
df_champion_players = pd.read_excel("Champions_Players.xlsx")  
df_player_ranks = pd.read_excel("Player_Ranks.xlsx")  
df_mjt = pd.read_excel("Michael_Jordan_Trophy.xlsx")  



df_players['Birth_date'] = pd.to_datetime(df_players['Birth_date'])
df_players['Birth_date'] = df_players['Birth_date'].dt.strftime('%Y-%m-%d')

df_players.drop(columns=["Death_date"], inplace=True)
if 'Unnamed: 0' in df_teams.columns:
    df_teams.drop(columns=['Unnamed: 0'], inplace=True)

In [13]:
df_players.to_sql('players', con=engine, if_exists='append', index=False)
df_positions.to_sql('positions', con=engine, if_exists='append', index=False)
df_teams.to_sql('teams', con=engine, if_exists='append', index=False)
df_player_positions.to_sql('player_positions', con=engine, if_exists='append', index=False)
df_player_teams.to_sql('player_teams', con=engine, if_exists='append', index=False)
df_seasons.to_sql('seasons', con=engine, if_exists='append', index=False)
df_champions.to_sql('champions', con=engine, if_exists='append', index=False)
df_champion_players.to_sql('champion_players', con=engine, if_exists='append', index=False)
df_player_ranks.to_sql('player_ranks', con=engine, if_exists='append', index=False)
df_mjt.to_sql('michael_jordan_trophy', con=engine, if_exists='append', index=False)

61

In [14]:
try:
    with mysql.connector.connect(**config) as cnxn:
        query = """
            SELECT p.Name, p.Height, s.Season
            FROM michael_jordan_trophy mjt
            JOIN players p ON mjt.PlayerID = p.PlayerID
            JOIN seasons s ON mjt.SeasonID = s.SeasonID
            WHERE s.Season IN ('2019-20','2020-21','2021-22','2022-23','2023-24')
            ORDER BY s.Season, p.Height DESC;
        """
        df_mjt_players = pd.read_sql(query, cnxn)
        
    display(df_mjt_players)

except mysql.connector.Error as err:
    print(f"Error: {err}")

C:\Users\Asus\AppData\Local\Temp\ipykernel_21676\2093797567.py:11: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_mjt_players = pd.read_sql(query, cnxn)


,Name,Height,Season
0,Giannis Antetokounmpo,211,2019-20
1,Nikola Jokić,211,2019-20
2,Anthony Davis,208,2019-20
3,LeBron James,206,2019-20
4,Pascal Siakam,203,2019-20
...,...,...,...
56,Jayson Tatum,203,2023-24
57,Luka Dončić,198,2023-24
58,Shai Gilgeous-Alexander,198,2023-24
59,Anthony Edwards,193,2023-24


In [15]:
try:
    with mysql.connector.connect(**config) as cnxn:
        query = """
            SELECT p.Name, p.Height, s.Season, pr.Rank
            FROM player_ranks pr
            JOIN players p ON pr.PlayerID = p.PlayerID
            JOIN seasons s ON pr.SeasonID = s.SeasonID
            WHERE s.Season >= '2019-20' AND pr.Rank <= 50
            ORDER BY s.Season, pr.Rank;
        """
        df_top50_height = pd.read_sql(query, cnxn)
        
    display(df_top50_height)

except mysql.connector.Error as err:
    print(f"Error: {err}")

C:\Users\Asus\AppData\Local\Temp\ipykernel_21676\2612962933.py:11: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_top50_height = pd.read_sql(query, cnxn)


,Name,Height,Season,Rank
0,James Harden,196,2019-20,1
1,Damian Lillard,188,2019-20,2
2,Devin Booker,198,2019-20,3
3,Giannis Antetokounmpo,211,2019-20,4
4,Trae Young,185,2019-20,5
...,...,...,...,...
295,Desmond Bane,196,2024-25,46
296,Evan Mobley,211,2024-25,47
297,Norman Powell,190,2024-25,48
298,Miles Bridges,201,2024-25,49


In [16]:
try:
    with mysql.connector.connect(**config) as cnxn:
        query = """
SELECT 
    p.Name,
    p.Height,
    p.Experience,
    s.Season
FROM champion_players cp
JOIN players p ON cp.PlayerID = p.PlayerID
JOIN seasons s ON cp.SeasonID = s.SeasonID
JOIN champions c ON cp.ChampionID = c.ChampionID
WHERE s.Season IN ('2023-24','2024-25')
ORDER BY s.Season, p.Height DESC;

        """
        championplayers = pd.read_sql(query, cnxn)
        
    display(championplayers)

except mysql.connector.Error as err:
    print(f"Error: {err}")

C:\Users\Asus\AppData\Local\Temp\ipykernel_21676\3040339517.py:17: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  championplayers = pd.read_sql(query, cnxn)


,Name,Height,Experience,Season
0,Luke Kornet,218,8,2023-24
1,Kristaps Porziņģis,218,9,2023-24
2,Neemias Queta,213,4,2023-24
3,Drew Peterson,206,2,2023-24
4,Al Horford,206,18,2023-24
5,Dalano Banton,206,4,2023-24
6,Jayson Tatum,203,8,2023-24
7,Xavier Tillman Sr.,203,5,2023-24
8,Sam Hauser,203,4,2023-24
9,Svi Mykhailiuk,201,7,2023-24


In [17]:
try:
    with mysql.connector.connect(**config) as cnxn:
        query = """
            SELECT p.Name, p.Height, p.Experience, s.Season, pr.Rank
            FROM player_ranks pr
            JOIN players p ON pr.PlayerID = p.PlayerID
            JOIN seasons s ON pr.SeasonID = s.SeasonID
            WHERE s.Season IN ('2024-25', '2023-24') AND pr.Rank <= 15
            ORDER BY pr.Rank;
        """
        top15 = pd.read_sql(query, cnxn)
        
    display(top15)

except mysql.connector.Error as err:
    print(f"Error: {err}")

C:\Users\Asus\AppData\Local\Temp\ipykernel_21676\485261729.py:11: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  top15 = pd.read_sql(query, cnxn)


,Name,Height,Experience,Season,Rank
0,Shai Gilgeous-Alexander,198,7,2024-25,1
1,Luka Dončić,198,7,2023-24,1
2,Anthony Edwards,193,5,2024-25,2
3,Shai Gilgeous-Alexander,198,7,2023-24,2
4,Giannis Antetokounmpo,211,12,2023-24,3
5,Nikola Jokić,211,10,2024-25,3
6,Jalen Brunson,188,7,2023-24,4
7,Giannis Antetokounmpo,211,12,2024-25,4
8,Jayson Tatum,203,8,2024-25,5
9,Nikola Jokić,211,10,2023-24,5


In [18]:
try:
    with mysql.connector.connect(**config) as cnxn:
        query = """
            SELECT p.Name, COUNT(mjt.TrophyID) AS Trophy_Count, p.PTS
            FROM michael_jordan_trophy mjt
            JOIN players p ON mjt.PlayerID = p.PlayerID
            JOIN player_positions pp ON p.PlayerID = pp.PlayerID
            JOIN positions pos ON pp.PositionID = pos.PositionID
            WHERE pos.PositionName = 'Point Guard'
            GROUP BY p.PlayerID
            ORDER BY Trophy_Count DESC;
        """
        pointguards = pd.read_sql(query, cnxn)
        
    display(pointguards)

except mysql.connector.Error as err:
    print(f"Error: {err}")

C:\Users\Asus\AppData\Local\Temp\ipykernel_21676\943130366.py:13: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pointguards = pd.read_sql(query, cnxn)


,Name,Trophy_Count,PTS
0,Giannis Antetokounmpo,5,23.9
1,Luka Dončić,5,28.6
2,Stephen Curry,3,24.7
3,LeBron James,3,27.0
4,Chris Paul,3,17.0
5,Jalen Brunson,2,18.1
6,Shai Gilgeous-Alexander,2,24.4
7,Ja Morant,2,22.6
8,Damian Lillard,2,25.1
9,James Harden,2,24.1


In [19]:
try:
    with mysql.connector.connect(**config) as cnxn:
        query = """
            SELECT 
                p.Name,
                (p.Height / p.Weight) AS Agility,
                s.Season,
                CASE 
                    WHEN s.Season IN ('2020-21','2021-22') THEN 'قدیم'
                    ELSE 'جدید'
                END AS Period
            FROM players p
            JOIN player_ranks pr ON p.PlayerID = pr.PlayerID
            JOIN seasons s ON pr.SeasonID = s.SeasonID
            WHERE pr.Rank <= 20 
              AND s.Season IN ('2020-21','2021-22','2022-23','2023-24')
            ORDER BY 
                Period,
                s.Season,
                Agility DESC;
        """
        top20 = pd.read_sql(query, cnxn)
        
    display(top20)

except mysql.connector.Error as err:
    print(f"Error: {err}")  

C:\Users\Asus\AppData\Local\Temp\ipykernel_21676\2557377413.py:22: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  top20 = pd.read_sql(query, cnxn)


,Name,Agility,Season,Period
0,Trae Young,2.5000,2022-23,جدید
1,Jalen Green,2.4125,2022-23,جدید
2,De'Aaron Fox,2.2892,2022-23,جدید
3,Jordan Poole,2.2184,2022-23,جدید
4,Shai Gilgeous-Alexander,2.2000,2022-23,جدید
...,...,...,...,...
75,Karl-Anthony Towns,1.9018,2021-22,قدیم
76,Anthony Edwards,1.8922,2021-22,قدیم
77,LeBron James,1.8230,2021-22,قدیم
78,Joel Embiid,1.6772,2021-22,قدیم


In [ ]:
try:
    with mysql.connector.connect(**config) as cnxn:
        query = """
            SELECT 
                p.Name,
                p.Experience,
                ROUND(DATEDIFF(DATE(CONCAT('20', RIGHT(s.Season, 2), '-06-30')), p.Birth_date) / 365.25, 1) AS Age_then,
                CASE 
                    WHEN p.Experience >= (2025 - (2000 + CAST(RIGHT(s.Season, 2) AS UNSIGNED))) 
                    THEN p.Experience - (2025 - (2000 + CAST(RIGHT(s.Season, 2) AS UNSIGNED)))
                    ELSE 0
                END AS Exp_then,
                CASE 
                    WHEN ROUND(DATEDIFF(DATE(CONCAT('20', RIGHT(s.Season, 2), '-06-30')), p.Birth_date) / 365.25, 1) > 0 
                    AND p.Experience >= (2025 - (2000 + CAST(RIGHT(s.Season, 2) AS UNSIGNED)))
                    THEN (p.Experience - (2025 - (2000 + CAST(RIGHT(s.Season, 2) AS UNSIGNED)))) / 
                        ROUND(DATEDIFF(DATE(CONCAT('20', RIGHT(s.Season, 2), '-06-30')), p.Birth_date) / 365.25, 1)
                    ELSE NULL
                END AS Skill,
                s.Season,
                CASE 
                    WHEN s.Season IN ('2022-23', '2023-24') THEN 'جدید'
                    ELSE 'قدیم'
                END AS Period
            FROM champion_players cp
            JOIN players p ON cp.PlayerID = p.PlayerID
            JOIN seasons s ON cp.SeasonID = s.SeasonID
            WHERE s.Season IN ('2020-21', '2021-22', '2022-23', '2023-24')
                AND p.Birth_date IS NOT NULL
                AND p.Experience IS NOT NULL
                AND DATEDIFF(DATE(CONCAT('20', RIGHT(s.Season, 2), '-06-30')), p.Birth_date) > 0
            ORDER BY Period, s.Season, Skill DESC;
        """
        skill = pd.read_sql(query, cnxn)
        
    display(skill)

except mysql.connector.Error as err:
    print(f"Error: {err}")

C:\Users\Asus\AppData\Local\Temp\ipykernel_21676\2917064973.py:20: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  skill = pd.read_sql(query, cnxn)


,Name,Experience,Age,Skill,Season,Period
0,Al Horford,18,39,0.4615,2023-24,جدید
1,Jrue Holiday,16,35,0.4571,2023-24,جدید
2,Jaylen Brown,9,28,0.3214,2023-24,جدید
3,Kristaps Porziņģis,9,30,0.3000,2023-24,جدید
4,Jayson Tatum,8,27,0.2963,2023-24,جدید
...,...,...,...,...,...,...
68,Bones Hyland,4,25,0.1600,2022-23,قدیم
69,Davon Reed,4,30,0.1333,2022-23,قدیم
70,Peyton Watson,3,23,0.1304,2022-23,قدیم
71,Christian Braun,3,24,0.1250,2022-23,قدیم
